## Example: Data set generation via CADENCE SPECTRE

In [1]:
import pandas as pd
import numpy as np
import serafin as sf
import pyspectre as ps

from scipy.stats import qmc
from functools import partial
from concurrent.futures import ThreadPoolExecutor

#### Create design space sample structure via a SOBOL sequence

In [2]:
expo        = 8

sampler     = qmc.Sobol(d=8, scramble=False)

sample      = sampler.random_base2(m=expo)

l_bounds    = [20e-6, 90e-9, 20e-6, 90e-9, 20e-6, 90e-9, 20e-6, 90e-9]
u_bounds    = [50e-6, 10e-6, 50e-6, 10e-6, 50e-6, 10e-6, 50e-6, 10e-6]

sample      = qmc.scale(sample, l_bounds, u_bounds)

sample.shape

(256, 8)

#### Generate random multipliers

In [3]:
a       = np.random.choice(5, 2**expo) + 1

b       = np.random.choice(9, 2**expo) + 2

col     = ['M']

dfm1    = pd.DataFrame(data = a, columns = col)

dfm2    = pd.DataFrame(data = b, columns = col)

#### Concatenate the sizings dataframe

In [4]:
cols        = ['Wcm1', 'Lcm1', 'Wcm2', 'Lcm2', 'Wcm4', 'Lcm4', 'Wdp1', 'Ldp1']
df          = pd.DataFrame(data = sample, columns = cols)

df['Wcm3']  = df['Wcm2'].values
df['Lcm3']  = df['Lcm2'].values
df['Mdp11'] = 2
df['Mdp12'] = 2
df['Mcm11'] = 1
df['Mcm12'] = dfm1.values
df['Mcm21'] = 1
df['Mcm22'] = dfm2.values
df['Mcm31'] = 1
df['Mcm32'] = dfm2.values
df['Mcm41'] = 2
df['Mcm42'] = 2

cols        = ['Ldp1', 'Lcm1', 'Lcm2', 'Lcm3', 'Lcm4', 'Wdp1', 'Wcm1', 'Wcm2', 'Wcm3', 'Wcm4', 'Mdp11', 'Mdp12', 'Mcm11', 'Mcm12', 'Mcm21', 'Mcm22', 'Mcm31', 'Mcm32', 'Mcm41', 'Mcm42']

df          = df[cols]

dfs         = np.array_split(df, 4)

df.head()

,Ldp1,Lcm1,Lcm2,Lcm3,Lcm4,Wdp1,Wcm1,Wcm2,Wcm3,Wcm4,Mdp11,Mdp12,Mcm11,Mcm12,Mcm21,Mcm22,Mcm31,Mcm32,Mcm41,Mcm42
0,9.000000e-08,9.000000e-08,9.000000e-08,9.000000e-08,9.000000e-08,0.000020,0.000020,0.000020,0.000020,0.000020,2,2,1,5,1,5,1,5,2,2
1,5.045000e-06,5.045000e-06,5.045000e-06,5.045000e-06,5.045000e-06,0.000035,0.000035,0.000035,0.000035,0.000035,2,2,1,5,1,8,1,8,2,2
2,7.522500e-06,2.567500e-06,2.567500e-06,2.567500e-06,7.522500e-06,0.000028,0.000043,0.000028,0.000028,0.000043,2,2,1,3,1,8,1,8,2,2
3,2.567500e-06,7.522500e-06,7.522500e-06,7.522500e-06,2.567500e-06,0.000043,0.000028,0.000043,0.000043,0.000028,2,2,1,2,1,6,1,6,2,2
4,8.761250e-06,3.806250e-06,8.761250e-06,8.761250e-06,1.328750e-06,0.000031,0.000031,0.000039,0.000039,0.000031,2,2,1,5,1,7,1,7,2,2


#### Define functions for data generation

In [5]:
pdk = '/home/moldenhauert/.circus/pdk/gpdk090.yml'
ckt = '/home/moldenhauert/.circus/ckt/sym.yml'
net = '/home/moldenhauert/.circus/pdk/gpdk090/sym.scs'

num = 32

def test_perf(op, sz):
    prf  = sf.evaluate(op,sz)

    return prf

def test_perf_parallel(tpe, syms, szs):
    pf = list(tpe.map(test_perf, syms, szs))
    return pf

def dataGen(k, df):
    with ThreadPoolExecutor(max_workers = num) as tpe:
        args  = zip(num * [pdk], num * [ckt], num * [net])
        syms  = list(tpe.map(lambda a: sf.operational_amplifier(*a), args))

    szs_list = np.array_split(df, len(df))

    sizes_df = pd.DataFrame()
    perfs_df = pd.DataFrame()

    with ThreadPoolExecutor(max_workers = num) as tpe:

        for i, chunk in enumerate([szs_list[i:i+num] for i in range(0,len(szs_list),num)]):
                    
            perfs = test_perf_parallel(tpe, syms, chunk)

            sizes_df = pd.concat([sizes_df, pd.concat(chunk)])
            perfs_df = pd.concat([perfs_df, pd.concat(perfs)])
    
    sizes_df = sizes_df.reset_index(drop=True)
    perfs_df = perfs_df.reset_index(drop=True)

    df1 = pd.concat([sizes_df, perfs_df], axis=1)

    filename = 'sobol_sizing_geometrical_gpdk090_' + str(k) + '.csv'

    df1.to_csv(filename, index = False)

#### Generate the performance values chunkwise

In [6]:
for i, chonk in enumerate(dfs):
    dataGen(i, chonk)
    print('Successfully generated chonk: ' + str(i))

Successfully generated chonk: 0
Successfully generated chonk: 1
Successfully generated chonk: 2
Successfully generated chonk: 3
